In [ ]:
!pip install python-dotenv

In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import pandas as pd

In [ ]:
load_dotenv('./.env')

False

In [ ]:
books_dir = './data'
documents = []
for filename in os.listdir(books_dir):
    if filename.endswith('.txt'):
        with open(os.path.join(books_dir, filename), 'r', encoding='utf-8') as file:
            text = file.read()
            documents.append(text)

print(f"Загружено {len(documents)} книг.")

Загружено 7 книг.


In [ ]:
BOOK_NAMES = {
    1: 'Гарри Поттер и философский камень',
    2: 'Гарри Поттер и тайная комната',
    3: 'Гарри Поттер и узник Азкабана',
    4: 'Гарри Поттер и кубок огня',
    5: 'Гарри Поттер и орден феникса',
    6: 'Гарри Поттер и принц-полукровка',
    7: 'Гарри Поттер и дары смерти',
}

In [ ]:
import re

def get_part_name(part):
    result = re.search(r'\d+\n+\s*(.*?)\n+', part, re.DOTALL)
    if result:
        extracted_text = result.group(1).strip()
        return extracted_text

In [ ]:
books_and_parts = {}
book_num = 0
for book in documents:
    book_num += 1
    parts = book.split('Глава ')
    book_parts = {}
    for num_part in range(1, len(parts)):
        book_parts = book_parts | {
            num_part: (get_part_name(parts[num_part]), parts[num_part])
        }
    books_and_parts = books_and_parts | {book_num: book_parts}

In [ ]:
books_and_parts

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

docs = []

text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n\n\n', '\n', '\n\n', "", " "],
    chunk_size=4000,
    chunk_overlap=500,
    length_function=len,
    is_separator_regex=False,
)

for book_num, parts in books_and_parts.items():
    for part_num, part in parts.items():
        texts = text_splitter.split_text(part[1])
        metadata = {
            'Номер книги': book_num,
            'Название книги': BOOK_NAMES[book_num],
            'Номер главы': part_num,
            'Название главы': part[0]
        }
        if part[0] is None:
            print(metadata)
        for text_part in texts:
            if len(text_part) < 300:
                continue
            docs.append(Document(page_content=f"{part[0]}\n{text_part}\n", metadata=metadata))


{'Номер книги': 4, 'Название книги': 'Гарри Поттер и кубок огня', 'Номер главы': 14, 'Название главы': None}


In [ ]:
!pip install faiss-cpu langchain_community

In [ ]:
!pip install sentence-transformers

In [ ]:
import faiss
from langchain.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain.embeddings import HuggingFaceEmbeddings


model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
index = faiss.IndexFlatL2(len(model.embed_query('hello world')))

vectorstore = FAISS(
    embedding_function=model,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
    distance_strategy='cosine'
)

vectorstore.add_documents(documents=docs)

<ipython-input-9-ffa446f157ee>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dataset

['361cd773-146c-45c6-baf7-70a84db4476f',
 '7cdef8fb-1627-4194-b4f8-fb040c9172b5',
 '4571d932-7fef-4d53-ae7b-ec1fdd849c73',
 '6ff03e25-9aa7-4093-a094-ecfe14a1418f',
 'c3e7740b-eb49-4d08-b812-4e0716737840',
 '15346d46-7222-4bbe-b03c-1d358c37c5b7',
 'e33e82fe-0697-440b-b443-24a0249f9868',
 'd6f33b86-924f-443c-ad6f-652ee46bc1c2',
 'f566e1f2-01b1-416f-85cc-5b4a03407358',
 'fb5635a1-82c4-4a53-8de1-c5a5501b78b9',
 '1561de2c-567b-42db-afc3-dc29d9ebb1a7',
 '9d4a2c7b-7503-450d-840c-1b813c478f72',
 'f6d66514-9f78-4eb3-94d1-50da4aca723b',
 'd15c285f-8406-4ae5-88df-28d18e4044b3',
 '119da841-89fc-4204-975b-9ecf9d6e9375',
 '4370fc2f-dd4a-4b88-8f03-64cc5a3f2877',
 '64c6b9aa-e50f-44a8-b82a-89b4a804c613',
 '689f85b1-11bc-4e70-9e44-3c8b3bc82c3a',
 '411f9cb4-b526-483b-82ac-46d1709ecda6',
 'c71008fa-e4cc-45db-ab11-a25a5f94672b',
 '7369e112-5383-43e1-ba6f-42e65268ea13',
 '36700300-f99c-43a1-83e1-017b9bb6bea5',
 'd4562d6a-1c19-4f87-91ed-25201e1049f7',
 'b50d9960-5c8a-4574-aff9-3e6710506d14',
 '3149af4a-a41d-

In [ ]:
vectorstore.save_local('HP_vector_store')

#### Замена модели для тестирования

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = 'EleutherAI/gpt-neo-125M' #'google/gemma-2-2b-it'
tokenizer = AutoTokenizer.from_pretrained(model_name)
hf_model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer.pad_token = tokenizer.eos_token

sys_prompt = '''
Ты часть RAG системы по поиску информации по книгам о Гарри Поттере.\n
Пользователь задает вопросы по сюжету/персонажам книги, ты предоставляешь ответ на основе базы знаний по книге, с указанием глав/частей книги, на основе которых был сформирован ответ.\n
Информация о частях книги тебе будет подаваться RAG составляющей, не забудь ее упомянуть при ответе.\n
Перед ответом проверяй себя - из-за неверных твоих ответов пользователь может не выжить, ты же не хочешь, чтобы кто-то из-за тебя умер.\n
Отвечай коротко, не повторяй полный текст RAG системы, только книгу, главу и ответ на вопрос.\n
'''

def answer(prompt):
    retriever = vectorstore.as_retriever()
    retr_res = retriever.invoke(prompt, k=3)

    print(retr_res)

    rag_content=''.join(res.page_content for res in retr_res)
    metadata_content=''.join(str(res.metadata) for res in retr_res)

    input_text = (
        f"{sys_prompt}\n"
        f"Запрос пользователя: {prompt}\n"
        f"Результаты RAG: {rag_content}\n"
        f"Использованные части: {metadata_content}\n"
    )

    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    )

    if inputs["input_ids"].max() >= tokenizer.vocab_size:
        raise ValueError(
            f"Некорректный токен в входных данных: максимум {inputs['input_ids'].max()}, "
            f"а словарь модели {tokenizer.vocab_size}."
        )

    outputs = hf_model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=50,
        pad_token_id=tokenizer.pad_token_id
    )

    answer_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer_text


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

#### Примеры выводов модели

In [ ]:
result = answer("Сколько лет было Гарри Поттеру в 5 книге?")
print(result)

[Document(id='5a175b53-79a6-4071-becf-d7b8abe6d3cd', metadata={'Номер книги': 7, 'Название книги': 'Гарри Поттер и дары смерти', 'Номер главы': 5, 'Название главы': 'КОСОЙ ПЕРЕУЛОК'}, page_content='КОСОЙ ПЕРЕУЛОК\n5\n   КОСОЙ ПЕРЕУЛОК\n   На следующее утро Гарри проснулся рано. Он знал, что уже рассвело, но не торопился открывать глаза.\n   «Это был сон,\xa0— твердо сказал он себе.\xa0— Мне приснилось, что ко мне приходил великан по имени Хагрид, чтобы сообщить мне, что я пойду учиться в школу волшебников. Когда я открою глаза, то окажусь дома в своем чулане».\n   Внезапно раздался громкий стук.\n   «А вот и тетя Петунья»,\xa0— подумал Гарри с замиранием сердца. Но глаза его все еще были закрыты. Сон был слишком хорош, чтобы просыпаться.\n   Тук. Тук. Тук.\n   —\xa0Хорошо,\xa0— пробормотал Гарри.\xa0— Я встаю.\n   Он сел, и тяжелая куртка Хагрида, под которой он спал, упала на пол. Хижина была залита светом, ураган кончился, Хагрид спал на сломанной софе, а на подоконнике сидела сова с

In [ ]:
result = answer("Какой номер был у платформы, с которой отправлялся поезд «Хогвартс Экспресс»?")
print(result)

[Document(metadata={'Номер книги': 2, 'Название книги': 'Гарри Поттер и тайная комната', 'Номер главы': 1, 'Название главы': 'СОВИНАЯ ПОЧТА'}, page_content='СОВИНАЯ ПОЧТА\n«Уважаемый мистер Поттер!\n   Напоминаю Вам, что первого сентября начинается учебный год. «Хогвартс-Экспресс» отходит от платформы № 9 и 3/4 с вокзала Кингс-Кросс в 11.00. В выходные дни третьекурсникам будет разрешено посещать деревню Хогсмид. К письму прилагается форма с разрешением. Ее должны подписать ваши родители или опекун. Также прилагается список учебников для третьего класса.\n   Искренне Ваша\n   профессор М. Макгонагалл,\n   заместитель директора школы».\n   Гарри достал форму с разрешением, и улыбки как не бывало. Дядя Вернон и тетя Петунья ни за что не подпишут эту бумагу. А было бы замечательно посещать Хогсмид по выходным. Это единственная деревня, где живут только волшебники, а он ни разу там не был!\n   Гарри посмотрел на будильник. Два часа ночи.\n   «Ладно, подожду до утра». С этими мыслями он лег

In [ ]:
result = answer("Как называется золотой мячик для игры в квиддич?")
print(result)

[Document(metadata={'Номер книги': 1, 'Название книги': 'Гарри Поттер и философский камень', 'Номер главы': 1, 'Название главы': 'МАЛЬЧИК, КОТОРЫЙ ВЫЖИЛ'}, page_content='МАЛЬЧИК, КОТОРЫЙ ВЫЖИЛ\n1\n   МАЛЬЧИК, КОТОРЫЙ ВЫЖИЛ\n   Мистер и миссис Дурсли проживали в доме номер четыре по Тисовой улице и всегда с гордостью заявляли, что они, к счастью, абсолютно нормальные люди. Уж от кого-кого, аот них никак нельзя было ожидать, чтобы они попали в какую-нибудь странную или загадочную историю. Мистер и миссис Дурсли весьма неодобрительно относились к любым странностям, загадкам и прочей ерунде.\n [Картинка: i_001.jpg] \n\n   Мистер Дурсль возглавлял фирму под названием «Граннингс», которая специализировалась на производстве дрелей. Это был полный мужчина с очень пышными усами и оченькороткой шеей. Что же касается миссис Дурсль, она была тощей блондинкой с шеей почти вдвое длиннее, чем положено при ее росте. Однако этот недостаток пришелся ей весьма кстати, поскольку большую часть времени мисс

### Тестирование с langsmith

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_ENDPOINT'] = "https://api.smith.langchain.com"
os.environ['LANGCHAIN_PROJECT'] = 'test_langchain'
os.environ["LANGCHAIN_API_KEY"] = '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'

In [ ]:
from langsmith import Client

client = Client()

examples = [
    ("Сколько лет было Гарри Поттеру в 5 книге?", "'В пятой книге, 'Гарри Поттер и Орден феникса', Гарри Поттеру было 15 лет. Это можно подтвердить разделами, в которых говорится о его возрасте. \n\n### Использованные части:\n- **Книга:** Гарри Поттер и Орден феникса\n- **Глава:** 26, 'Секреты из прошлого' (не указана в предоставленных данных, но это тот раздел, где обсуждается его возраст)\n\nЕсли требуется больше информации или детали, дайте знать!'"),
    ("Какой номер был у платформы, с которой отправлялся поезд «Хогвартс Экспресс»?", "СОВИНАЯ ПОЧТА «Уважаемый мистер Поттер! Напоминаю Вам, что первого сентября начинается учебный год. «Хогвартс-Экспресс» отходит от платформы № 9 и 3/4 с вокзала Кингс-Кросс в 11.00. В выходные дни третьекурсникам будет разрешено посетить в ответ на основе книги. После этого не при"),
]

dataset_name = "Agent RAG Queries Example"
dataset = client.create_dataset(dataset_name=dataset_name)
inputs, outputs = zip(
    *[({"question": text}, {"ground_truth": label}) for text, label in examples]
)
client.create_examples(inputs=inputs, outputs=outputs, dataset_id=dataset.id)

In [ ]:
%%capture --no-stderr
!pip install langsmith langchain langchain-community langchain-chroma langchain-openai ragas

In [ ]:
from langsmith.schemas import Example, Run

def correct_answer(root_run: Run, example: Example) -> dict:
    score = root_run.outputs.get("output") == example.outputs.get("answer")
    return {"score": int(score), "key": "correct_answer"}

In [ ]:
from langsmith.evaluation import evaluate

dataset_name = "Agent RAG Queries Example"

results = evaluate(
    lambda inputs: answer(inputs["question"]),
    data=dataset_name,
    evaluators=[correct_answer],
    experiment_prefix="Agent RAG Queries",
    description="Testing the baseline system.",  # optional
)

View the evaluation results for experiment: 'Agent RAG Queries-89798cbb' at:
https://smith.langchain.com/o/426a7616-6d62-504b-ae5b-b0131d82108a/datasets/971d1c4d-7ecf-438a-b680-db2a7438cc3b/compare?selectedSessions=b7eca8fe-afb3-4721-a05d-dbb0ba3e66b4




0it [00:00, ?it/s]

[Document(id='78ca7695-b1e0-488d-ba05-ea1e9186eb12', metadata={'Номер книги': 6, 'Название книги': 'Гарри Поттер и принц-полукровка', 'Номер главы': 1, 'Название главы': 'СОВИНАЯ ПОЧТА'}, page_content='СОВИНАЯ ПОЧТА\n«Уважаемый мистер Поттер!\n   Напоминаю Вам, что первого сентября начинается учебный год. «Хогвартс-Экспресс» отходит от платформы № 9 и 3/4 с вокзала Кингс-Кросс в 11.00. В выходные дни третьекурсникам будет разрешено посещать деревню Хогсмид. К письму прилагается форма с разрешением. Ее должны подписать ваши родители или опекун. Также прилагается список учебников для третьего класса.\n   Искренне Ваша\n   профессор М. Макгонагалл,\n   заместитель директора школы».\n   Гарри достал форму с разрешением, и улыбки как не бывало. Дядя Вернон и тетя Петунья ни за что не подпишут эту бумагу. А было бы замечательно посещать Хогсмид по выходным. Это единственная деревня, где живут только волшебники, а он ни разу там не был!\n   Гарри посмотрел на будильник. Два часа ночи.\n   «Ла